El objetivo de este notebook es cargar los archivos csv de la tasa objetivo, INPC y tipo de cambio para limpiarlos, transformar las fechas a data_time y poner las fechas como indices. Ademas, limpiar los datos y ajustar los datos diarios para homologarlos a mensuales. 

In [ ]:
import pandas as pd 
import os
import sys


#Nos permite subir de nivel para acceder a otras carpetas
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from banxico_utils import convertirDF
#Guardar la direccion de la carpeta donde estan nuestros archivos     
data_dir = os.path.join(project_root, 'data')
print(f"Buscando datos en: {data_dir}")

#Construimos las rutas finales a los CSV
ruta_tasa = os.path.join(data_dir, "TasaObjetivo.csv")
ruta_INPC = os.path.join(data_dir, "INPCGeneral.csv")
ruta_tipo_cambio = os.path.join(data_dir, "TipoDeCambio.csv")




#Cargar archivos csv
try:
    df_TasaObjetivo=pd.read_csv(ruta_tasa)
    df_INPC=pd.read_csv(ruta_INPC)
    df_TipoCambio=pd.read_csv(ruta_tipo_cambio)
    print("Archivos cargados con exito; DF creados.")
except FileNotFoundError:
    print("Error al cargar archivos")
    
    
#Llamar a funcion para convertir fechas 
df_TasaObjetivoI=convertirDF(df_TasaObjetivo)
df_INPCI=convertirDF(df_INPC)
df_TipoCambioI=convertirDF(df_TipoCambio)

#Imprimir DF solo para comprobar que se hayan aplicado los cambios a los DF
print(df_TasaObjetivoI.head(5))
print(df_INPCI.head(5))
print(df_TipoCambioI.head(5))
    
    

El siguiente paso es conseguir que todos los datos queden en una serie de tiempo mensual. Como cada dato tiene una naturaleza diferente, se tratarán de distintas formas.
Tambien haremos una concatenacion para guardar los tres df en un unico df


In [ ]:
#Usar resample para INPC que ya es mensual, pero queremos que se asigne al ultimo dia de dicho mes
df_INPCm=df_INPCI.resample("M").mean()
#Usar resample para tipo de cambio
df_TipoCambiom=df_TipoCambioI.resample("M").mean()
#Usar resample para tasa objetivo
df_TasaObjetivom=df_TasaObjetivoI.resample("M").asfreq().ffill()

#Crear nuestro dataframe maestro concatenando los tres df
listaDF=[df_INPCm,df_TipoCambiom,df_TasaObjetivom]

df_maestro=pd.concat(listaDF,axis=1,join="outer",ignore_index=False).dropna()
print(df_maestro.head(5))

"""""
#Crear ruta para guardar el dataframe maestro 
ruta_dfMaestro=os.path.join(data_dir, "DataFrameMaestro.csv")

#Guardar en archivo csv
df_maestro.to_csv(ruta_dfMaestro,index=False)
"""